### **LLM API 활용해보기**

In [4]:
from dotenv import load_dotenv
import os

# 1. 현재 경로의 .env 파일 로드
load_dotenv()

# 2. 환경 변수 읽기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

**[오픈AI의 GPT-3.5 Turbo 모델 API 호출 코드]**

In [5]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": "Who won the world series in 2020?"
        }
    ]
)

ChatCompletion(id='chatcmpl-CSYtD3lzU40rwjanvMVEAWto9F4Tb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The Los Angeles Dodgers won the World Series in 2020, defeating the Tampa Bay Rays. This victory marked the Dodgers' first championship since 1988.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760923335, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=32, prompt_tokens=17, total_tokens=49, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

**[랭체인을 활용한 오픈AI GPT-3.5 Turbo 모델 API 호출 코드]**

In [6]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name='gpt-4o-mini',
    openai_api_key=OPENAI_API_KEY
)
chat.invoke("안녕~ 너를 소개해줄래?")

AIMessage(content='안녕하세요! 저는 AI 언어 모델 ChatGPT입니다. 다양한 질문에 답하고, 정보 제공, 글쓰기 지원, 대화 등 여러 가지 작업을 도와드릴 수 있어요. 궁금한 점이나 필요한 것 있으면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 17, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CSYwUQJY3vkyUn0UgiFatGHho8JjL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--cf899ded-cd82-4a4a-94ec-a336377642ee-0', usage_metadata={'input_tokens': 17, 'output_tokens': 56, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### **프롬프트 템플릿에 대해 알아보기**

**[ChatPromptTemplate]**

In [7]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
	#SystemMessage: 유용한 챗봇이라는 역할과 이름을 부여
        ("system", "You are a helpful AI bot. Your name is {name}."), 
    #HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
    #HumanMessage로 사용자가 입력한 프롬프트를 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


In [8]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
       "You are a helpful assistant that re-writes the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I don't like eating tasty things", additional_kwargs={}, response_metadata={})]


### **LLM API의 다양한 기능 활용해보기**

**[LLM API의 Temperature 이애하기]**

In [9]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#OPENAI API키 저장
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

#Temperature=0
chatgpt_temp0_1 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)
chatgpt_temp0_2 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

#Temperature=1
chatgpt_temp1_1 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)
chatgpt_temp1_2 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)

model_list = [chatgpt_temp0_1, chatgpt_temp0_2, chatgpt_temp1_1, chatgpt_temp1_2]

for i in model_list:
    answer = i.invoke("왜 파이썬이 가장 인기있는 프로그래밍 언어인지 한 문장으로 설명해줘", max_tokens = 128)
    print("-"*100)
    print(">>>",answer.content)

----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법, 다양한 라이브러리와 프레임워크, 그리고 데이터 과학, 웹 개발, 인공지능 등 다양한 분야에서의 활용성 덕분에 가장 인기 있는 프로그래밍 언어로 자리잡았습니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법, 다양한 라이브러리와 프레임워크, 그리고 데이터 과학, 웹 개발, 인공지능 등 다양한 분야에서의 활용성 덕분에 가장 인기 있는 프로그래밍 언어로 자리잡았습니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법 덕분에 배우기 쉽고, 다양한 분야에서 활용 가능한 라이브러리와 프레임워크를 제공하여 널리 인기를 끌고 있습니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 직관적인 문법, 광범위한 라이브러리와 프레임워크, 그리고 다양한 분야에서의 활용 가능성 덕분에 가장 인기 있는 프로그래밍 언어로 자리잡고 있습니다.


**[ChatGPT처럼 답변 스트리밍하기]**

In [10]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)
for chunk in chat.stream("달에 관한 시를 써줘"):
    print(chunk.content, end="", flush=True)

**달빛의 노래**

밤하늘에 걸린 은빛 구슬,  
조용히 속삭이는 달의 노래.  
어둠 속에 피어나는 꿈들,  
그대의 미소가 내 마음을 감싸네.

구름 사이로 얼굴을 내밀고,  
별들과 함께 춤추는 그 모습.  
차가운 바람에 실려오는  
그리움의 향기, 달빛의 속삭임.

나무 사이로 스며드는 빛,  
흩날리는 잎사귀에 담긴 이야기.  
어릴 적 그리던 환상 속에서,  
달은 언제나 나의 친구였네.

이 밤, 고요한 마음으로 바라보니,  
세상의 모든 슬픔이 사라지고.  
달빛 아래, 나의 소원 하나,  
영원히 함께하길, 그대와 나.

**[답변 캐싱하기]**

In [11]:
from langchain.globals import set_llm_cache #캐시메모리 라이브러리 호출
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

**첫 질문-응답 시간 측정**

In [12]:
%%time 
#셀 실행 시간 측정
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache()) #캐시메모리 설정

chat.invoke("일반상대성 이론을 한마디로 설명해줘")

CPU times: total: 172 ms
Wall time: 1.54 s


AIMessage(content='일반상대성 이론은 중력이 시공간의 곡률로 설명되며, 질량이 큰 물체가 주변의 시공간을 휘게 만들어 다른 물체의 운동에 영향을 미친다는 이론입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CSZ0sdX7yFlNVQYoO1Fkr88wVHWFT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--02f97738-bfdf-4b81-bd52-9f979f6dd7c8-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

**두번째 질문-응답 시간 측정**

In [13]:
%%time
#같은 질문 전달
chat.invoke("일반상대성 이론을 한마디로 설명해줘")

CPU times: total: 0 ns
Wall time: 290 μs


AIMessage(content='일반상대성 이론은 중력이 시공간의 곡률로 설명되며, 질량이 큰 물체가 주변의 시공간을 휘게 만들어 다른 물체의 운동에 영향을 미친다는 이론입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CSZ0sdX7yFlNVQYoO1Fkr88wVHWFT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--02f97738-bfdf-4b81-bd52-9f979f6dd7c8-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}, 'total_cost': 0})

### **실습**

**[이번 장에서 배운 것 실습해보기] - 스트리밍되는 AI스터디 플래너 챗봇 만들기**

In [14]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

#GPT-3.5 모델 호출
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)

#ChatPromptTemplate 통해 스터디 플래너 역할 부여 및 사용자 프롬프트 매개변수화
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "당신은 공부 계획을 세워주는 스터디 플래너 머신입니다."
                "사용자의 공부 주제를 입력 받으면, 이를 학습하기 위한 공부 계획을 작성합니다."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

#앞서 설정한 프롬프트 템플릿에 HumanMessage로 문장 전달
messages = chat_template.format_messages(text="Large Language Model에 대해서 공부하고 싶어요.")

#stream 함수를 통해 답변 스트리밍
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

Large Language Model (LLM)에 대한 공부 계획을 세워드리겠습니다. 이 계획은 4주 동안 진행되며, 매주 주제를 나누어 학습할 수 있도록 구성하였습니다.

### 1주차: LLM의 기초 이해
- **목표**: LLM의 기본 개념과 역사 이해
- **학습 내용**:
  - LLM의 정의 및 특징
  - 자연어 처리(NLP)의 기본 개념
  - LLM의 발전 역사 (예: GPT, BERT 등)
- **추천 자료**:
  - 온라인 강의: Coursera, edX의 NLP 기초 강의
  - 관련 논문: "Attention is All You Need" (Transformer 모델 소개)
- **실습**: 간단한 NLP 프로젝트 (예: 텍스트 분류)

### 2주차: LLM의 구조와 작동 원리
- **목표**: LLM의 내부 구조와 작동 원리 이해
- **학습 내용**:
  - Transformer 아키텍처의 구성 요소 (Self-Attention, Feedforward Network 등)
  - 학습 과정 (Pre-training vs Fine-tuning)
  - LLM의 평가 지표 (Perplexity, BLEU Score 등)
- **추천 자료**:
  - 블로그 포스트: Jay Alammar의 "The Illustrated Transformer"
  - 관련 논문: "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"
- **실습**: Hugging Face Transformers 라이브러리 설치 및 기본 모델 사용해보기

### 3주차: LLM의 응용 및 활용
- **목표**: LLM의 다양한 응용 분야 이해
- **학습 내용**:
  - LLM의 활용 사례 (챗봇, 텍스트 생성, 번역 등)
  - LLM의 한계와 윤리적 고려사항
  - Fine-tuning을 통한 특정 작업에 대한 모델 조정
- **추천 자료**:
  - 관련 논문: "Lang